#### Menu Driven Program with Collaborative Filtering Using the Movies and Users Dataset 

In [1]:
import pandas as pd
import numpy as np

In [2]:
def menu():
    print("\nThe Menu:")
    print("a. Count User")
    print("b. Count Movies")
    print("c. Display User Information")
    print("d. Display Movie Name")
    print("e. Recommend Movie")
    print("f. Exit")
    option=input("Option: ")
    return option

def countUsers(user_dict):
    return len(user_dict.keys())

def countMovies(movie_dict):
    return len(movie_dict.keys())

def displayUser(user_dict):
    userid=int(input("User ID: "))
    print(pd.DataFrame(user_dict[userid],columns=["Movie ID","Rating","Timestamp"]))

def displayMovieName(movieid,movie_dict):
    return movie_dict[movieid][0]

def calculate_similarity(userid, user_dict):
    similarities = {}
    target_user_ratings = {movie[0]: movie[1] for movie in user_dict[userid]}

    for other_user_id, other_user_ratings in user_dict.items():
        if other_user_id == userid:
            continue
        
        other_user_dict = {movie[0]: movie[1] for movie in other_user_ratings}
        
        common_movies = set(target_user_ratings.keys()).intersection(set(other_user_dict.keys()))
        
        if not common_movies:
            continue
        
        target_vector = [target_user_ratings[movie] for movie in common_movies]
        other_vector = [other_user_dict[movie] for movie in common_movies]
        
        dot_product = sum(a * b for a, b in zip(target_vector, other_vector))
        target_magnitude = sum(a ** 2 for a in target_vector) ** 0.5
        other_magnitude = sum(b ** 2 for b in other_vector) ** 0.5
        
        if target_magnitude == 0 or other_magnitude == 0:
            continue
        
        similarity = dot_product / (target_magnitude * other_magnitude)
        similarities[other_user_id] = similarity
    
    return similarities

def recommend_movies(userid, user_dict, num_recommendations=5):
    similarities = calculate_similarity(userid, user_dict)
    
    weighted_ratings = {}
    similarity_sums = {}

    for other_user_id, similarity in similarities.items():
        other_user_ratings = user_dict[other_user_id]
        
        for movie_id, rating, _ in other_user_ratings:
            if movie_id not in {movie[0] for movie in user_dict[userid]}:
                if movie_id not in weighted_ratings:
                    weighted_ratings[movie_id] = 0
                    similarity_sums[movie_id] = 0
                weighted_ratings[movie_id] += rating * similarity
                similarity_sums[movie_id] += similarity

    recommendations = {}
    for movie_id, total_weight in weighted_ratings.items():
        if similarity_sums[movie_id] > 0:
            recommendations[movie_id] = total_weight / similarity_sums[movie_id]
    
    recommended_movies = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)
    
    return [movie_id for movie_id, score in recommended_movies[:num_recommendations]]

In [3]:
movie_df=pd.read_csv('./Movie_Id_Titles.csv')
user_df=pd.read_csv('./user_dataset.csv')

movie_dict={}
user_dict={}

for i in sorted(movie_df.item_id.unique()):
    movie_dict[i]=movie_df[movie_df['item_id']==i].drop('item_id',axis=1).to_numpy()[0]

for i in sorted(user_df.user_id.unique()):
    user_dict[i]=user_df[user_df['user_id']==i].drop('user_id',axis=1).to_numpy()

option='a'
while(option in {'a','b','c','d','e'}):
    option=menu()
    if(option=='a'):
        print(f"Number of Users = {countUsers(user_dict)}")
    elif(option=='b'):
        print(f"Number of Movies = {countMovies(movie_dict)}")
    elif(option=='c'):
        displayUser(user_dict)
    elif(option=='d'):
        movieid=int(input("Movie ID: "))
        print(displayMovieName(movieid,movie_dict))
    elif(option=='e'):
        userid=int(input("User ID: "))
        print("Recommended Movies: ")
        for i in recommend_movies(userid,user_dict,10):
            print("\t",movie_dict[i])


The Menu:
a. Count User
b. Count Movies
c. Display User Information
d. Display Movie Name
e. Recommend Movie
f. Exit
